Using AmgX to Accelerate PETSc-Based CFD Codes
==================================================

Pi-Yueh Chuang  
George Washington University

## Intorduction
-------------------

In this talk, I want to share how we accelerate our PETSc-based CFD code, PetIBM, with AmgX library and GPUs.

PetIBM is a parallel CFD program for CPUs. We use PETSc library to help parallelization. And now we want to accelerate it with the help of GPU computing. The first try is our another CFD code, cuIBM, which is almost the same with PetIBM, except that it uses CUSP library and totally lives on one single GPU. While memory on a single GPU is limited, and hence cuIBM can not handle larger CFD problems, we realized that we need to use multiple GPUs. But we have already spent so much time developing PetIBM and cuIBM, we probably can not afford another half year or even one year to develop a new program or rewrite PetIBM just for migrating it from CPU to multi-GPU platforms. We need a library to help us the multi-GPU capability, and we also need to do this with least coding effort in PetIBM but best resulting performance. 

The library we choosed is AmgX, which enables the multi-GPU capability. And we write a wrapper to coupling AmgX and PETSc, so that we won't have to do many modifications in PetIBM.

This will be our story today. We hope our story will help you, if you also have some PETSc-based parallel programs and want to accelerate them with multiple GPUs.

## A brief introduction of PetIBM
-----------------------------------------

We all know solving Poisson problem is diffucult. Especially for unsteady CFD, it is very tough because we need to solve the Poisson system again and again during time-marching stage. But the situation in PetIBM is much tougher than normal unsteady CFD programs since we are not just solving Poisson equations, we are solving a so-called ***modified Poisson equation***. Let's take a look at PetIBM to see what that is.

In PetIBM, we use Taira and Colonius' method [1]. The following is the governing PDE we solve in PetIBM:

$$
\left\{
\begin{aligned}
& \frac{\partial \bf{u}}{\partial t} + \bf{u}\cdot\nabla\bf{u} = 
-\nabla\it{p}+\frac{1}{Re}\nabla^2\bf{u}+\int_s{\bf{f}(\bf{\xi}(\it{s}, \it{t}))\delta(\bf{\xi}-\bf{x})}d\it{s}, \\
& \nabla\cdot\bf{u}=0\it{,} \\
& \bf{u}(\bf{\xi}(\it{s}, \it{t})) = 
\int_\bf{x}{\bf{u}(\bf{x})\delta(\bf{x}-\bf{\xi})}d\bf{x}=\bf{u}_\it{B}(\bf{\xi}(\it{s}, \it{t}))
\end{aligned}
\right.
$$

If we write the above equations into a block system of linear equations, in which each submatrix represents an operator or a term in the PDE:

$$
\left[
\begin{matrix}
A & G & E^T \\
G^T & 0 & 0 \\
E & 0 & 0
\end{matrix}
\right]
\left(
\begin{matrix}
q^{n+1} \\
\phi \\
\tilde{f}
\end{matrix}
\right)
=
\left(
\begin{matrix}
r^n \\
0 \\
u_B^{n+1}
\end{matrix}
\right)
+
\left(
\begin{matrix}
bc_1 \\
-bc_2 \\
0
\end{matrix}
\right)
$$

We can further define some larger submatrices and perform a block LU decomposition:

$$
Q\equiv\left[G, E^T\right],\ 
\lambda\equiv\left(\begin{matrix}\phi \\ \tilde{f}\end{matrix}\right),\ 
r_1\equiv r^n + bc_1,\ 
r_2\equiv\left(\begin{matrix}-bc_2 \\ u_B^{n+1}\end{matrix}\right)
$$

$$
\Rightarrow
\left[
\begin{matrix}
A & 0 \\
Q^T & -Q^TB^NQ
\end{matrix}
\right]
\left[
\begin{matrix}
I & B^NQ \\
0 & I
\end{matrix}
\right]
\left(
\begin{matrix}
q^{n+1} \\
\lambda
\end{matrix}
\right)
=
\left(
\begin{matrix}
r_1 \\
r_2
\end{matrix}
\right)
+
\left(
\begin{matrix}
-\frac{\Delta t^N}{2^N}\left(LM^{-1}\right)Q\lambda \\
0
\end{matrix}
\right)
$$

We rewrite the above equation:

$$
\begin{aligned}
& Aq^*=r_1 \\
& Q^TB^NQ\lambda=Q^Tq^*-r_2 \\
& q^{n+1}=q^*-B^NQ\lambda
\end{aligned}
$$

The middle equation is the one we called ***modified Poisson equation***, which is a Poisson equation plus some modification terms. The coefficient matrix in the modified Poisson eqauation, $Q^TB^NQ$, has the following non-zero pattern:

<img src="plots/cylinder3dRe40QTBNQ.png" style="width:300px;height:300px;" />

The upper left part in the coefficient matrix represent the coefficients of Poisson equation, and at the bottom, bottom right corner and the right side are those modification terms.

This kind of pattern makes it difficult to solve. In PetIBM, solving this modified Poisson equation takes over ***90%*** of total time. So this gives us an alternative choice to accelerate PetIBM if we don't want to rewrite the whole porgram: we can introduce GPU computing into this part and accelerate only this part.

## AmgX
------------

Now we have determined where in PetIBM we want to introduce GPU computing into. Next one is what GPU library we want to use. The one we choosed is AmgX.

AmgX is a library providing linear solvers and preconditioners with capability of multiple GPUs. It is developed and supported by NVIDIA. This means whenever users have problems, they can always find someone to help them.

Here we list some features of AmgX:

* Krylov methods:
    * CG, GMRES, BiCGStab, ... etc
* Multigrid preconditioners
    * Classical AMG
    * Unsmoothed aggregation AMG
* Smoothers
    * Block-Jacobi, Gauss-Seidel, incomplete LU, dense LU, ... etc
* Cycles:
    * V, W, F, FG, CGF
* Multiple GPUs on single node / multiple nodes
    * MPI (OpenMPI) / MPI Direct
    * Single MPI rank <=> single GPU
    * Multiple MPI ranks <=> single GPU
* C API
* Unified Virtual Addressing

The combinations of Krylov methods and multigrid preconditioners are very popular in CFD applications. This is the first reason why we choose AmgX. The second reason is of course the multi-GPU support.

If you are interested in this library, you can find it and its tutorial, manual and license from [this website](https://developer.nvidia.com/amgx)[2]. 

## AmgX wrapper
-----------------------

So far we've determined the library we are going to use, which is AmgX, and also where in PetIBM we will introduce AmgX. Now we want to write a wrapper. Why?

First, AmgX and PETSc have their own data structures for parallel linear algebra. If we want to use AmgX directly in PetIBM, we will need to add or modify a lot of code in it, which is not preferable. PetIBM is a mature program. And we know that if we do more modifications in a mature program, it is more prone to bugs. This is not what we want to see. So we want to do as few modifications as possible in PetIBM.

Second, in future, we may develop some other software using PETSc, and we may also want to use AmgX in them. If we write a wrapper to couple AmgX and PETSc now, it will help a lot in the future.

Actually there are some other minor reasons, such as: PetIBM is an object-oriented programming, but AmgX API is pure C; and some other reasons. But they are all minors. 

So, we write a wrapper which is independent to the applications.

The pholosophy of our wrapper is simple: except initialization and finalization, users only need to call `setA` and `solve` in most cases.

<img src="plots/wrapper.png" style="width:500px;height:300px;" />

For unsteady CFD applications, during time marching stage, users can call `solve` repeatedly. `A`, `x`, and `rhs` are all PETSc data structures. No need to concern about data conversion. And configurations can be done through an input script during runtime.

## First Test: 2D Cylinder Flow, Re=40
-----------------------------------------------

So far, the wrapper is just a wrapper. It's just an interface between PETSc ans AmgX and should work well. Let's see how it works. First we run tests with a 2D cylinder flow (Re=40). Mesh size is 2.25M.

<img src="plots/rawAmgXMultiRanks_Cylinder2d_2.png" style="width:500px;height:500px;" />

The far left bar is the result from original CPU-version PetIBM with 6 CPU cores. We can see that solving modified Poisson equation, which is the blue portion, takes over 90% of total time in this case. 

The middle bar is the result from using the AmgX-version PetIBM with 1 GPU and 1 CPU core. We get 4 times speed-up on solving modified Poisson equation and 3 times speed-up for total time. We think it looks great, but not great enough. From the parts that are solved on CPUs, which are the portions with colors other than blue, we find now it takes more time to solve them. The reason is obvious: in this case we only use 1 CPU core to solve these CPU-based parts, while 6 CPU cores were used to solve them in the far left bar. Let's see how if we use 6 CPU cores to solve the CPU-based parts and 1 GPU to solve the modified Poisson system.

The far right bar is the result. We indeed get the same performances on the parts solved or calculated with CPUs, but the speed-up of solving modified Poisson equation is not good as we expected. Actually, it's totally unacceptable for me. So, what happened?

It's not difficult to answer it. Just think about what will happen if you solve a linear system with domain-decomposition methods and partition the system into 6 subdomains but solve them with only one CPU core or less than 6 CPU cores. Situation here is the same.

Recall that AmgX solvers only solve the linear system. That means we need to prepare the linear systems on CPUs and using PETSc. So when we launch PetIBM with 6 CPU cores, the modified Poisson system is partitioned into 6 subdomains through some partitioning methods in PETSc. And when we call AmgX to solve the modified Poisson system, each CPU core creates an AmgX solver to solve the subdomain it holds. In our case, there are 6 subdomain-solvers but only one GPU. This means the 6 subdomain-solvers are competing GPU resources. And remember we are using domain-decomposition methods, so information exchanges occurs between these 6 subdomain-solvers and on one single GPU. Solving modified Poisson equation in the far right bar therefore is not as fast as we expected.

## More examples revealing this issue:
-------------------------------------------------
Let's see more examples revealing this issue:

#### 1. Solving 3D Poisson equations (6M unknowns; CG solver; Classical AMG preconditioner):

<img src="plots/rawAmgXMultiRanks_Classical.png" style="width:450px;height:300px;" />


#### 2. Solving modified Poisson equations from cylinder flow simulations (6M unknowns; CG solver; Aggregation AMG preconditioner)

<img src="plots/rawAmgXMultiRanks_Aggregation.png" style="width:450px;height:300px;" />

Both cases show that when we prepare linear systems with PETSc and more CPU processes than the number of GPUs we used, the performances of GPU solvers get worse.

## Solution
--------------

Solution is not complicated. We just need to prepare the linear systems that AmgX is going to solve with the same number of subdomains as that of GPUs. The purpose is to have only one subdomain-solver on each GPU.

If we are going to modified the source code of PetIBM, there are plenty of ways to reach the goal. No matter using which approach, we can not avoid doing a lot modifications, because while we want to use only the same number of CPU processes as that of GPUs to handle the modified Poisson equation, we still want to use as many CPU processes as possible to calculate and solve the parts other than modified Poisson equation, like the velocity system, for example.

So, given that the reasons why we write a wrapper are that we want to make using AmgX easy and that we want to do as few modifications as possible in PetIBM, we should implement the solution to this issue into the wrapper, instead of directly modify the source code of PetIBM.

The concept is demonstrated in this figure:

<img src="plots/mergeSysOnHosts.png" style="width:450px;height:300px;" />

The implementation is achieved through communicator splitting. We briefly illustrate this step by step:

##### Step 1. Split the global communicator (eg. MPI_COMM_WORLD) into communicators representing each node. 
<table border="0" style="width:100%">
  <tr>
    <td style="border-style: hidden; border-collapse: collapse;"><img src="plots/commSplit1.png" style="width:400px;height:200px;" /></td>
    <td style="border-style: hidden; border-collapse: collapse;">$\Longrightarrow$</td> 
    <td style="border-style: hidden; border-collapse: collapse;"><img src="plots/commSplit2.png" style="width:400px;height:200px;" /></td>
  </tr>
</table>
When we launch a program, we know how many CPU processes, computing node, and the number of GPUs on each node, but the program doesn't. Without additional coding, the program only knows the total number of CPU processes. So the first step is to split the global communicator, which is typically `MPI_COMM_WORLD` into communicators representing each computing node. Then we can calculate how many GPU devices there are on each node. 

##### Step 2. Devide the CPU processes equaly according to the number of GPUs on each node
<img src="plots/commSplit3.png" style="width:400px;height:200px;" />
So that for each node, the number of CPU processes sharing one GPU is the same with those sharing another GPU on the same node. This is also done with communicator splitting. Each ***matrix-consolidation communicator*** represents CPU processes that share the same GPU. And their data, or says, the subdomains they hold, will be consolidated and gatherred before being transferred to GPUs. 

##### Step 3. Assign one CPU process in each matrix-consolidation communicator to be in charge of launching AmgX subdomain-solver
<img src="plots/commSplit4.png" style="width:400px;height:200px;" />
Normally, they are rank zero in each matrix-consolidation communicator. Only these CPU processes can talk to GPUs.

##### Step 4. Create a new communicator that holds all CPU processes that can talk to GPUs
<img src="plots/commSplit5.png" style="width:400px;height:200px;" />
Because we are solving linear equations using domain-decomposition methods, AmgX subdomain-solvers on GPUs need to talk to each others. Since GPUs can not talk to each others directly, their communications are done through CPUs and MPI. So we need to create a communicator for the CPUs that can talk to GPUs, and then they can talk to each others, too.

Not only consolidation of the coefficient matrix is handled in the wrapper, the gathering and scattering of unknown and right-han-side vectors are also handled in the wrapper, of course.

We implement this in the wrapper, and this keeps the usage of AmgX and the wrapper simple in PetIBM or other PETSc-based applications. We still only need to call `setA` and `solve` and don't need to worry about other else. 

## Check whether this is worth or not
--------------------------------------------------

From my previous brief illustration, you can find there are some extra communications we need in order to do matrix consolidation or gather and scatter vectors. Communications take time. So we need to do some tests to see whether this is worth or not.

#### 1. Back to the 3D Poisson system

<img src="plots/newAmgXMultiRanks_Classical.png" style="width:450px;height:300px;" />

#### 2. Back to the modofied Poisson systems

<img src="plots/newAmgXMultiRanks_Aggregation.png" style="width:450px;height:300px;" />

We can see now no matter how many CPU processes we used to prepare the linear system, the times used to solve the system on GPU with AmgX and wrapper are almost the same.

#### 3. Back to the 2D cylinder flow

<img src="plots/newAmgXMultiRanks_Cylinder2d_2.png" style="width:500px;height:500px;" />

The far right bar now is the result from PetIBM with AmgX and host-side consolidation enabled in the wrapper. Now the time used to solve the modified Poisson equation, which is the blue portion, is the same as the middle bar. And we can see, from the portions with colors other than blue, now we can use as many CPU cores as possible. This means, with our wrapper, users can utilize all CPU and GPU resources they have to achieve the best performance by just calling `setA` and `solve` in their existing PETSc-based programs.

## Tests and benchmarks
---------------------------------

Since our wrapper is ready, we can do some tests and benchmarks to see how the performance of AmgX is. As I said, our goal here is to accelerate our CFD programs, PetIBM, so our tests focus on two problems. The first one is Poisson equations, which represents the flows without solid body present, like lid-driven flows. The second one is modified Poisson equation. For this one, we will directly use PetIBM to run a real application and see how much speed-up we can get.

### 1. Poisson Equation

##### a. Small-size problem

<table border="0" style="width:100%">
  <tr>
    <td style="border-style: hidden; border-collapse: collapse;"><img src="plots/Poisson_N1M2D.png" style="width:450px;height:275px;" /></td>
    <td style="border-style: hidden; border-collapse: collapse;"><img src="plots/Poisson_N1M3D.png" style="width:450px;height:275px;" /></td>
  </tr>
</table>

Well, it seems AmgX doesn't benefit a lot in both 2D and 3D problems.

In CPU parts, blue bars represent the results of using PETSc KSP solvers and PETSc gamg preconditioners, while purple bars are those using Hypre preconditioners but still with PETSc KSP solvers. In GPU parts, red bars are the results from GPU clusters in HPC center of our university, and the green bars are the results from a workstation in our lab. By having both results from GPU clusters and GPU workstation, we want to see if it is possible that, when we have GPU computing, even a workstation can be competitive to traditional CPU clusters.

Since the result in small-size problem seems not as good as we expected, we want to see whether this is because the problem size is too small.

##### b. Medium-size problem

<table border="0" style="width:100%">
  <tr>
    <td style="border-style: hidden; border-collapse: collapse;"><img src="plots/Poisson_N25M2D.png" style="width:450px;height:275px;" /></td>
    <td style="border-style: hidden; border-collapse: collapse;"><img src="plots/Poisson_N25M3D.png" style="width:450px;height:275px;" /></td>
  </tr>
</table>

Now we have 25 millions unknowns in our problem, which might not be unusual in aerodynamics or hydrodynamics industry. 

In CPU parts, we take off the results from GAMG preconditioners, because it is much slower than Hypre in this case. And in GPU parts, we need at least 4 and 8 K20 GPU in 2D and 3D problems respectively because of limited memory on single K20. Also, since we only have 2 K40 on our workstation, so the workstation can not handle the 3D case.

I don't know what you think, but for me, I think the result is not bad.

In the 2D case, since the CPU results seem to have good scalability, we can predict that, if we want to achieve 5 times speed-up, we may need probably 100 CPU cores. But if we use AmgX, we only need 4 K20. I will say four K20 is better than 100 CPU cores. Because we can actually have 4 K20 on our personal workstation, an no need to use clusters. Also, don't forget K20 is kind of an old model. You can probably find it on Amazon at a price of 25-hundred dollars for a new one. So, 4 K20 may cost 10 thousands, while I don't think 10 thousand can give you a CPU cluster with 100 CPU cores.

And in 3D case, again, if we want to achieve 13.6 times speed-up by only CPU computing, we may need 256 CPU cores, while we only need 8 K20 when we use AmgX. I'll say 8 K20 is better than 256 CPU cores. The same reason as I just illustrated in the 2D case. Though it may not be possible to hold 8 K20 in a personal workstation, but it is possible to install 8 K20 in an existing node in HPC clusters. Also, K20 is an old model, if we use K40 or K80, we can probably still use our personal workstation to compete against 256-CPU-core cluster.

##### c. Large-size problem

<table border="0" style="width:100%">
  <tr>
    <td style="border-style: hidden; border-collapse: collapse;"><img src="plots/Poisson_N100M2D.png" style="width:450px;height:275px;" /></td>
    <td style="border-style: hidden; border-collapse: collapse;"><img src="plots/Poisson_N50M3D.png" style="width:450px;height:275px;" /></td>
  </tr>
</table>

Though results of medium-size Poisson problem are good, I want to show you the results from large-size problem, just in case some of you are interested in simulations in this scale.

For the 2D case, in which we have 100 millions unknowns, we can predict that we may need almost 400 CPU cores to achieve the same speed-up as 32 GPUs. For the 3D case, we can only do 50 millions unknowns because the limited memory we have on all our GPUs. We may also need approximate 400 CPU cores to achieve the same speed-up.

And again, K20 is a very new model. So if we can have newer model, like K80, which has larger memory, we may need only few of them to achieve the same performance.

### 2. Flying Snake

For PetIBM, Poisson equation represents flows without solid body present, which is normally not real cases we simulate with. PetIBM is for flows passing some bodies, so solving modified Poisson equation is more common. Here, we use a real application we did with PetIBM as an example to demonstrate how AmgX can accelerate solving modified Poisson equation and hence accelerate the whole simulation.

Simulations of flying snakes are done by one of our group member, Anush. You can find details of flying snake in his 2014 paper[3].

<img src="plots/flyingSnake_Vorticity2.png" style="width:450px;height:275px;" />

We re-run one of the flying snake simulations with CPU cluster, K20 GPU cluster, and a personal workstation with 2 K40 GPU. Here are short specs of the hardware:

1. 1 node in CPU cluster = 12 CPU cores (2 Intel E5-2620)
2. 1 node in GPU cluster = 1 CPU node (i.e. 12 CPU cores) + 2 K20 GPU
3. personal workstation = 6 CPU cores (1 Intel i7-5930K) + 2 K40c GPU

You can find computing nodes in the GPU cluster are actually nodes in the CPU cluster with additional two K20 installed on each node. This setting can help us to determine whether using AmgX and GPU computing is beneficial from the viewpoint of money cost on hardware.

<img src="plots/flyingSnake.png" style="width:800px;height:600px;" />

If we use the result of 1 CPU node as our baseline, then the speed-up of using 1 GPU node is almost 21 times. Again, based on the scalability of CPU results, we can predict that we may need 20 CPU nodes in order to achieve 21 times speed-up if we are using pure CPU code. Recall one GPU node is actually 1 CPU node plus 2 K20. I think it's obvious that adding 2 K20 to an existing computing node is cheaper than adding 19 extra computing nodes.

Next let's move on to the results of using the workstation. I think the result here is much more exciting. When we use our personal workstation and one K40 GPU, we can get about 15 times speed-up, which can also be achieved with 16 CPU nodes in the CPU cluster. This means, at least for our applications, if the GPU memory on our workstation is large enough, we don't need to run our simulations on clusters. Our personal workstation can handle it.

So from these results, the benefit of AmgX and GPU computing for our CFD code, PetIBM, is very obvious. 

## More discussion: time is money
---------------------------------------------

So far we have mostly focused on the saving of time that AmgX can benefits us. But is that all? What else can those benchmarks imply?

We all know time is money. So when AmgX saves time for us, it also saves money for us. I want to discuss a little bit more here.

### 1. Potential saving on hardware

I mentioned a little bit about this when I showed the results to you just now. For example, from the flying snake simulation, using one workstation and one K40 GPU is competitive with a CPU cluster of 16 nodes. And of course using workstation can save you a lot. If you want to build a HPC cluster, you will need to consider costs on hardware like motherboards, memory, hard drives, cooling systems, power supplies, network switches, physical spaces ... etc. But now we only need to buy two GPUs and install them onto PCI-E slots of an existing workstation. 

Furthermore, if we can avoid building clusters, it does not only reduce the cost on hardware, but also reduce the cost on human resources, because you probably won't need to hire an IT guy to manage your cluster for you, or you won't need to add more work loading to your group members.

Image that when you want to promote your CFD software to customers who have limited budget on hardware but want to get
results quickly because of rapid developing cycle, would you prefer recommending them to build a cluster or recommending them to buy several GPUs and install them on existing workstation?

Of course we may still need clusters sometimes because of limited memory on GPUs and limited PCI-E slots on a single motherboard. But from our benchmark results, we can see that, with AmgX, even when we need clusters, we need much fewer computing nodes. This implies a lot, such as: less socket-to-socket communication; reducing potential runtime crash due to single-node failure or network failiure; or you can get higher priority in queue if you run simulations in HPC centers, because now you need less time and less nodes.

### 2. Potential saving on cloud HPC service

I also want to discuss about the benefit AmgX and GPU computing can give us when we use cloud HPC service, such as Amazon EC2.

We run benchmarks on Amazon EC2 to see what we can get. I list the specs of GPU nodes and CPU nodes we used on Amazon EC2 here:

<table border="0" width=100%>
    <tr>
        <td style="border-style: hidden; border-collapse: collapse;" width=50%>
            <ul> 
                <li> 
                    GPU nodes - g2.8xlarge:  
                    <ul>
                        <li> 32 vCPU (Intel E5-2670) + 4 GPUs (Kepler GK104 ) </li>
                        <li> Official Price: &#36;2.6 / hr </li>
                        <li> Possible Lower Price (Spot Instances): < &#36;0.75 / hr </li>
                    </ul>
                </li>
            </ul>
        </td>
        <td style="border-style: hidden; border-collapse: collapse;" width=50%>
            <ul> 
                <li> 
                    CPU nodes - c4.8xlarge
                    <ul>
                        <li> 36 vCPU (Intel E5-2663) </li>
                        <li> Official Price: &#36;1.675 / hr </li>
                        <li> Possible Lower Price (Spot Instances): < &#36;0.6 / hr </li>
                    </ul>
                </li>
            </ul>

        </td>
</tr>
</table>

Let's see the results:

<img src="plots/Amazon.png" style="width:500px;height:450px;\" />

Even we use 8 CPU nodes, we can still get 3 times speed-up on 1 GPU node! It's kind of surprising but not so surprising. The reason why it is surprising is because 8 CPU nodes have a total of 288 CPU cores! Well, but it is actually not so surprising when we consider that the nodes are virtual machines, not physical machines. We don't know what's inside Amazon's black box.

But one thing we can know is how much we can save!  First let's calculate the actual fee we paid for the simulations. We use spot instances, so the real fees we paid for nodes are:

* CPU nodes:
$$12.5hr \times \$0.5/hr \times 8nodes={\underline {\bf \$50.0}}$$
* GPU nodes:
$$4hr \times \$0.5/hr \times 1nodes={\underline {\bf \$2.0}}$$

Wow, we save 25 times on the cost!

Well, sometimes it's difficult to get spot instances or we are not willing to use them. Then, how much we can save if we rent the instances with official price? 

* CPU nodes:
$$12.5hr \times \$1.675/hr \times 8nodes={\underline {\bf \$167.5}}$$
* GPU nodes:
$$4hr \times \$2.6/hr \times 1nodes={\underline {\bf \$10.4}}$$

We still get about 16 times saving!

## Conclusion
-------------------

To conclude, we use AmgX to accelerate our CFD code, PetIBM. And we also write a wrapper to couple AmgX and PETSc for the purpose of easily using AmgX. Our wrapper can handle the situation that we want to launch the PETSc-based program with more MPI processes than the number of GPU we have. So with wrapper, we can utilize as many CPU processes as possible to run the parts still remain on CPU, and use as many GPU as possible to accelerate solving tough parts.

You can download AmgX, our wrapper, and PetIBM with multi-GPU computing enabled from:

* AmgX: https://developer.nvidia.com/amgx
* AmgX Wrapper: https://github.com/barbagroup/AmgXWrapper
* PetIBM: https://github.com/barbagroup/PetIBM/tree/AmgXSolvers

We also do some benchmarks and see how AmgX help us to save not only time but also money costs, no matter from the viewpoints of hardware or cloud HPC service. 

## Acknowledgement
----------------------------

I would like to give very special thanks to Dr. Joe Eaton from NVIDIA. He gave us a lot of help on this project.

#### Reference
-----------------

[1] K.  Taira and T.  Colonius, "The immersed boundary method: A projection approach", Journal of Computational Physics, vol. 225, no. 2, pp. 2118-2137, 2007

[2] AmgX, https://developer.nvidia.com/amgx

[3] A.  Krishnan, J.  Socha, P.  Vlachos and L.  Barba, "Lift and wakes of flying snakes", Physics of Fluids, vol. 26, no. 3, p. 031901, 2014